# Tipologia y ciclo de vida de los datos.PRACTICA 1. Web Scraping

El objetivo de esta practica es scrapear una website de venta de ordenadores y crear una base de datos con los ordenadores resultantes. Almacenando sus diferentes caracterisiticas: precio, marca, modelo, valoracion........

In [3]:
!pip install bs4 

In [4]:
#importo las librerias que necesitare

import urllib2
import urllib
import bs4
import re
from bs4 import BeautifulSoup as soup

In [5]:
!pip install python-whois

In [6]:
#veamos a quien pertencen los datos
import whois
print whois.whois('https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001')

{
  "updated_date": [
    "2015-03-03 13:42:09", 
    "2015-03-02 14:46:50"
  ], 
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited", 
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited", 
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited", 
    "serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited", 
    "serverTransferProhibited https://icann.org/epp#serverTransferProhibited", 
    "serverUpdateProhibited https://icann.org/epp#serverUpdateProhibited", 
    "clientUpdateProhibited (https://www.icann.org/epp#clientUpdateProhibited)", 
    "clientTransferProhibited (https://www.icann.org/epp#clientTransferProhibited)", 
    "clientDeleteProhibited (https://www.icann.org/epp#clientDeleteProhibited)", 
    "serverUpdateProhibited (https://www.icann.org/epp#serverUpdateProhibited)", 
    "serverTransferProhibited (https://www.icann.org/epp#serverTransferProhibited)", 
    "serverDeleteProhi

In [7]:
#urllib2 es un modulo de Python que usare para "agarrar" los URLs.Beautifulsoup me sirve para diseccionar texto html

In [8]:
#from urllib2.Request import urlopen as uReq, en principio no lo necesito, bastara con llamar el apquete y despues los modulos
#por ejemplo  request = urllib2.Request(url,headers=headers)


In [9]:
my_url = "https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001"

uClient = urllib2.urlopen(my_url)  #abro una conexion para agarrar la pagina
page_html = uClient.read()  #vacio el contenido de la web en una variable
uClient.close() 
#llamo a la funcion BeatifulSoup con la pagina que hemos descargado y le decimos que diseccione como html 
page_soup = soup(page_html, "html.parser")

In [10]:
#los siguientes 3 comandos son para ver si funciona y realmente estoy accediendo a la web, llamemos en la pagina de bestbuy 
#aquella parte que tenga la etiqueta h1 
page_soup.h1

<h1 class="type-page-label-alt ">All Laptops</h1>

In [11]:
page_soup.p

<p>Bring superior sound to your home theater.</p>

In [12]:
page_soup.div

<div class="skip-to-last-filter-container"></div>

In [13]:
#efectivamente esta funcionando y aque cada vez que llamamos a una etiqueta nos da un resultado

In [14]:
#Ahora convertiremos cada laptop en una linea de un csv file, guardando cada etiqueta div con la clase igual a "list-item".
#Despues de inspeccionar la web con el inspect de google chrome o el Firebug Lite, me doy cuenta que todo lo que quiero descargar
#esta en una etiqueta div donce la clase es igual a "list-item. Todo lo que esta contenido en ese container contiene toda la 
#informacion que deseo scrapear: precio, modelo, marca, sku......... 
#Como tengo varios ordenadores, tendre varios ordenadores tendre varios containers identicos, con lo cual buscare todos los 
#containers con div y class ="list-item" ayudandome de la funcion findAll, una vez encontrados los guardare en una variable que 
#llamare container. Cada elemento de container es un ordenador.

In [15]:
containers =  page_soup.findAll("div" ,{"class":"list-item"})
len(containers)   #veamos cuantos ordenadores ha encontrado por pagina

24

In [10]:
containers[0] #echemos un vistazo al codigo del primer elemento (primer ordenador) del paquete containers

<div class="list-item" data-activated-device="" data-attributes='{"newProduct":true}' data-availability='{"pickup":{"available":true,"availabilityCode":"115","checkStoresLink":{"url":"/site/olstemplatemapper.jsp?id=pcat17006&amp;type=page&amp;sourceType=product&amp;skuId=6097909&amp;pageMode=searchmode"},"displayMessage":"\\u003cb&gt;Store Pickup:\\u003c/b&gt;"},"ship":{"available":true,"checkLink":{"height":470,"target":"popup","url":"/site/olspage.jsp?id=cat13503&amp;type=page&amp;skuId=6097909","width":530},"displayMessage":"\\u003cb&gt;Shipping:\\u003c/b&gt;  Usually ships in 1 business day","getItBy":{"currentPrice":529.99,"availabilityCode":"110","regionalAvailabilityEligible":true}}}' data-availability-type="" data-average-rating="3.4" data-brand='{"brand":"Dell"}' data-bundle-items="" data-button-state='{"purchasable":true,"buttonState":"ADD_TO_CART","displayText":"Add to Cart","skuId":"6097909"}' data-clearance-puck="" data-condition="new" data-functional-map="" data-img-path=

In [11]:
print containers[0].prettify() #con prettyfy le damos un poco mas de estilo, para identificar mas facilmente aquello que quiero
#extraer

<div class="list-item" data-activated-device="" data-attributes='{"newProduct":true}' data-availability='{"pickup":{"available":true,"availabilityCode":"115","checkStoresLink":{"url":"/site/olstemplatemapper.jsp?id=pcat17006&amp;type=page&amp;sourceType=product&amp;skuId=6097909&amp;pageMode=searchmode"},"displayMessage":"\u003cb&gt;Store Pickup:\u003c/b&gt;"},"ship":{"available":true,"checkLink":{"height":470,"target":"popup","url":"/site/olspage.jsp?id=cat13503&amp;type=page&amp;skuId=6097909","width":530},"displayMessage":"\u003cb&gt;Shipping:\u003c/b&gt;  Usually ships in 1 business day","getItBy":{"currentPrice":529.99,"availabilityCode":"110","regionalAvailabilityEligible":true}}}' data-availability-type="" data-average-rating="3.4" data-brand='{"brand":"Dell"}' data-bundle-items="" data-button-state='{"purchasable":true,"buttonState":"ADD_TO_CART","displayText":"Add to Cart","skuId":"6097909"}' data-clearance-puck="" data-condition="new" data-functional-map="" data-img-path="Bes

In [12]:
#veamos el texto que contiene
for item in containers:
    print item.text

Add to CompareNew! Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - BlackModel: I3567-5664BLK-PUSSKU: 60979093.4(41)$529.99Included Free: 1 item Add to Cart
Add to CompareHP - 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 1TB Hard Drive - HP finish in jet blackModel: 15-BS015DXSKU: 58687054.3(1,131)$449.99On SaleSave $80Was $529.99Included Free: 1 item Add to CartOpen-Box from $417.99Open-Box from $417.99Open-BoxAs low as$417.99See All OptionsBuy new: $449.99
Add to CompareDell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i3 - 8GB Memory - 1TB Hard Drive - BlackModel: I3567-3636BLK-PUSSKU: 58358524.3(1,028)$399.99Included Free: 1 item Add to CartOpen-Box from $375.99Open-Box from $375.99Open-BoxAs low as$375.99See All OptionsBuy new: $399.99
Add to CompareMicrosoft - Surface Pro Core M with Black Type Cover - PlatinumModel: HGG-00001SKU: 6111206See More Options4.8(4)$999.00Included Free: 1 item Add to Cart
Add to CompareApple - M

In [13]:
#Nos da una gran cantidad de informacion pero no es muy manejable, hagamoslo mas manejable
for item in containers:
    print item.contents[0]

<div class="row"><div class="col-xs-3 list-item-thumbnail-column"><div class="list-item-thumbnail"><div class="thumb"><a data-rank="pdp" href="/site/dell-inspiron-15-6-touch-screen-laptop-intel-core-i5-8gb-memory-2tb-hard-drive-black/6097909.p?skuId=6097909"><img alt='Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black - Larger Front' class="center-block" src="https://pisces.bbystatic.com/image2/BestBuy_US/images/products/6097/6097909_sa.jpg;maxHeight=145;maxWidth=222" srcset="https://pisces.bbystatic.com/image2/BestBuy_US/images/products/6097/6097909_sa.jpg;maxHeight=290;maxWidth=444 2x"/></a></div></div><div class="list-item-compare-container"><div class="list-item-compare checkbox accessible-toggle"><label for="add-to-compare-6097909"><input autocomplete="off" class="list-item-compare-checkbox sr-only" id="add-to-compare-6097909" type="checkbox"><div aria-hidden="true" class="checkbox-brand"><i class="custom-input"></i></div>Add to Compare

In [14]:
#Para darle un poco mas de visibilidad al container[0] de arriba he usado una de las herramientas online para ver la estructura
#del codigo html anterior con cierto orden: https://www.danstools.com/javascript-beautify/

#La expresion de mas abajo es el container que incluye toda la informacion que queremos extraer de cada ordenador, este ejemplo 
#es para el 'Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black', su estructura nos 
#dara la base para poder generalizar a traves de loops y poder extraer informacion de cualquier laptop en el web que queramos
#scrapear.

In [ ]:
<div class="list-item" data-activated-device="" data-attributes='{"newProduct":true}' data-availability='{"pickup":{"available":true,"availabilityCode":"115","checkStoresLink":{"url":"/site/olstemplatemapper.jsp?id=pcat17006&amp;type=page&amp;sourceType=product&amp;skuId=6097909&amp;pageMode=searchmode"},"displayMessage":"\\u003cb&gt;Store Pickup:\\u003c/b&gt;"},"ship":{"available":true,"checkLink":{"height":470,"target":"popup","url":"/site/olspage.jsp?id=cat13503&amp;type=page&amp;skuId=6097909","width":530},"displayMessage":"\\u003cb&gt;Shipping:\\u003c/b&gt;  Usually ships in 1 business day","getItBy":{"currentPrice":359.99,"availabilityCode":"110","regionalAvailabilityEligible":true}}}' data-availability-type="" data-average-rating="3.1" data-brand='{"brand":"Dell"}' data-bundle-items="" data-button-state='{"purchasable":true,"buttonState":"ADD_TO_CART","displayText":"Add to Cart","skuId":"6097909"}' data-clearance-puck="" data-condition="new" data-functional-map="" data-img-path="BestBuy_US/images/products/6097/6097909_sa.jpg" data-install-dcs-whitelisted="" data-listing-id="" data-name='Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black' data-pickup-available="true" data-price="359.99" data-price-json='{"currentPrice":359.99,"pricingType":"onSale","regularPrice":529.99,"savingsAmount":170,"smartPricerEnabled":true,"priceDomain":{"skuId":"6097909","saleEventMessageType":"ONSALE","regularPriceMessageType":"WAS","regularPrice":529.99,"currentPrice":359.99,"customerPrice":359.99,"instantSavings":170,"totalSavings":170,"totalSavingsPercent":32,"isMAP":false,"offerQualifiers":[{"offerId":"134664","offerName":"Value Computers / Free GWP - S2","offerVersion":246418,"id":134664000}],"giftSkus":[{"skuId":"1050005154","quantity":1,"offerId":"134664"}],"currentAsOfDate":"2017-11-11T19:16:14 GMT","pricingType":"onSale","savingsAmount":170,"regularPriceMessageLabel":"Was $529.99"}}' data-pricing-type="onSale" data-product-id="" data-product-type="hardgood" data-properties='{"format":"physical","servicePlansType":"GSP","servicePlanDetailsURL":"/site/geek-squad-protection/computer-tablet-protection/pcmcat1489429207479.c?id=pcmcat148942920747"}' data-refurbished-puck="" data-review-count="25" data-sale-puck="" data-seller-id="" data-seller-info='{"seller":"BestBuy"}' data-show-medium-item-price-unactivated="" data-show-notify-me-button="" data-show-puck-product-options="true" data-sku-id="6097909" data-title='Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black' data-url="/site/dell-inspiron-15-6-touch-screen-laptop-intel-core-i5-8gb-memory-2tb-hard-drive-black/6097909.p?skuId=6097909">
    <div class="row">
        <div class="col-xs-3 list-item-thumbnail-column">
            <div class="list-item-badge">
                <div data-react-checksum="178812892" data-reactid="1" data-reactroot="" style="height:20px;overflow:hidden;position:relative;width:160px;"><img alt="Black Friday Ad Deal" data-reactid="2" src="https://pisces.bbystatic.com//image2/BestBuy_US/store/ee/2017/global/general/GL-17248-badge-adleak-171108.png" style="position:absolute;top:50%;left:50%;transform:translate(-50%, -50%);" />
                </div>
            </div>
            <div class="list-item-thumbnail">
                <div class="thumb">
                    <a data-rank="pdp" href="/site/dell-inspiron-15-6-touch-screen-laptop-intel-core-i5-8gb-memory-2tb-hard-drive-black/6097909.p?skuId=6097909"><img alt='Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black - Larger Front' class="center-block" src="https://pisces.bbystatic.com/image2/BestBuy_US/images/products/6097/6097909_sa.jpg;maxHeight=145;maxWidth=222" srcset="https://pisces.bbystatic.com/image2/BestBuy_US/images/products/6097/6097909_sa.jpg;maxHeight=290;maxWidth=444 2x" />
                    </a>
                </div>
            </div>
            <div class="list-item-compare-container">
                <div class="list-item-compare checkbox accessible-toggle">
                    <label for="add-to-compare-6097909">
                        <input autocomplete="off" class="list-item-compare-checkbox sr-only" id="add-to-compare-6097909" type="checkbox">
                        <div aria-hidden="true" class="checkbox-brand"><i class="custom-input"></i>
                        </div>Add to Compare</input>
                    </label>
                </div>
            </div>
        </div>
        <div class="col-xs-6 list-item-postcard-column">
            <div class="list-item-postcard">
                <div class="sku-title" itemprop="name">
                    <h4><a data-rank="pdp" href="/site/dell-inspiron-15-6-touch-screen-laptop-intel-core-i5-8gb-memory-2tb-hard-drive-black/6097909.p?skuId=6097909"><span class="new-product">New!</span> Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black</a></h4>
                </div>
                <div class="sku-model">
                    <ul>
                        <li class="model-number"><span class="sku-label">Model:</span> <span class="sku-value" id="model-value-6097909" itemprop="model">I3567-5664BLK-PUS</span>
                        </li>
                        <li class="sku-id"><span class="sku-label">SKU:</span> <span class="sku-value" id="sku-value-6097909" itemprop="productID">6097909</span>
                        </li>
                    </ul>
                </div>
                <div class="customer-rating" itemprop="aggregateRating" itemscope="" itemtype="http://schema.org/AggregateRating">
                    <meta content="1" itemprop="worstRating">
                    <meta content="5" itemprop="bestRating">
                    <meta content="25" itemprop="reviewCount">
                    <a aria-label="Rating: 3.1 out of 5 Stars with 25 reviews" href="/site/dell-inspiron-15-6-touch-screen-laptop-intel-core-i5-8gb-memory-2tb-hard-drive-black/6097909.p?skuId=6097909#tabbed-customerreviews">
                        <div class="c-ratings-reviews v-small" data-react-checksum="-103358297" data-reactid="1" data-reactroot="">
                            <p class="sr-only" data-reactid="2"></p><span alt="62%" aria-hidden="true" class="pl-stars pl-stars-small" data-reactid="3"><span class="unfilled" data-reactid="4"></span><span class="filled" data-reactid="5" style="width:62%;"></span></span><span class="c-reviews" data-reactid="6"><span aria-hidden="true" class="c-review-average" data-reactid="7">3.1</span><span aria-hidden="true" class="c-total-reviews" data-reactid="8"><!-- react-text: 9 -->(<!-- /react-text --><!-- react-text: 10 -->25<!-- /react-text --><!-- react-text: 11 --><!-- /react-text --><!-- react-text: 12 -->)<!-- /react-text --></span></span>
                        </div>
                    </a>
                    </meta>
                    </meta>
                    </meta>
                </div>
                <div class="availability-status-container lsa-block" id="lsa-174f67d0-c715-11e7-8a04-b7f5f9e7717d"></div>
                <div class="open-box-view"></div>
            </div>
        </div>
        <div class="col-xs-3 list-item-price-column">
            <div class="list-item-price">
                <div class="list-item-price-content">
                    <div class="list-item-price-spinner"></div>
                    <div class="priceblock-wrapper">
                        <div data-dux-component="priceViewTranslatedDataDux6097909">
                            <div class="price-view-pb" data-css="https://assets.bbystatic.com/price-view/price-view-91e8616bb4f109fe13fe.css" data-js="https://assets.bbystatic.com/price-view/price-view-91e8616bb4f109fe13fe.js" data-pricing="%7B%22pricing%22%3A%7B%22skuId%22%3A%226097909%22%2C%22options%22%3A%7B%22dux%22%3Atrue%2C%22openBox%22%3Atrue%2C%22activatedDux%22%3Atrue%2C%22bundleAndSave%22%3Atrue%2C%22priceMatchGuarantee%22%3Atrue%2C%22promoClassic%22%3Afalse%2C%22promoWhitespaceFix%22%3Afalse%2C%22layout%22%3A%22medium%22%2C%22context%22%3A%22PLP%22%2C%22rzTierCode%22%3A%22%22%7D%2C%22savings%22%3A%7B%22total%22%3A170%7D%2C%22regularPrice%22%3A%7B%22abbreviated%22%3A%22Was%20%24529.99%22%2C%22descriptive%22%3A%22The%20price%20was%20%24529.99%22%7D%2C%22isPriceMatchGuarantee%22%3Atrue%2C%22saleMessage%22%3A%22On%20Sale%22%2C%22purchasePrice%22%3A359.99%2C%22giftSkus%22%3A%5B%7B%22skuId%22%3A%221050005154%22%2C%22quantity%22%3A1%7D%5D%2C%22summarySkus%22%3A%5B%221050005154%22%5D%7D%2C%22options%22%3A%7B%22dux%22%3Atrue%2C%22openBox%22%3Atrue%2C%22activatedDux%22%3Atrue%2C%22bundleAndSave%22%3Atrue%2C%22priceMatchGuarantee%22%3Atrue%2C%22promoClassic%22%3Afalse%2C%22promoWhitespaceFix%22%3Afalse%2C%22layout%22%3A%22medium%22%2C%22context%22%3A%22PLP%22%2C%22rzTierCode%22%3A%22%22%7D%7D" data-sku-id="6097909" data-version="17.43.6"></div>
                        </div>
                        <div data-dux-component="priceViewPriceDux6097909">
                            <div class="price-view-pb" data-sku-id="6097909" data-version="17.43.6" data-view-type="price">
                                <div class="price-view-pb pb-layout-medium" data-react-checksum="1753433303" data-reactid="1" data-reactroot="">
                                    <div class="pb pb-price" data-reactid="2">
                                        <div class="pb-price-match-guarantee" data-reactid="3">
                                            <div class="pb-anchor" data-reactid="4">
                                                <a aria-label="Price Match Guarantee" data-reactid="5" href="javascript:void(0)">
                                                    <div aria-label="Price Match Guarantee" class="icon-price-match-gurantee" data-reactid="6"></div>
                                                </a>
                                            </div>
                                        </div>
                                        <div class="pb-hero-price pb-purchase-price" data-reactid="7"><span aria-label="Your price for this item is $359.99" data-reactid="8"><!-- react-text: 9 -->$<!-- /react-text --><!-- react-text: 10 -->359.99<!-- /react-text --></span>
                                        </div>
                                        <div class="pb-sale-message" data-reactid="11">On Sale</div>
                                        <div class="pb-savings" data-reactid="12">
                                            <!-- react-text: 13 -->Save $170
                                            <!-- /react-text -->
                                            <!-- react-text: 14 -->
                                            <!-- /react-text -->
                                            <!-- react-text: 15 -->
                                            <!-- /react-text -->
                                        </div>
                                        <div aria-label="The price was $529.99" class="pb-regular-price" data-reactid="16">Was $529.99</div>
                                    </div>
                                </div>
                            </div>
                        </div>
                        <div data-dux-component="priceViewFreeItemDux6097909">
                            <div class="price-view-pb" data-sku-id="6097909" data-version="17.43.6" data-view-type="free-item">
                                <div class="price-view-pb pb-layout-medium" data-react-checksum="-571308604" data-reactid="1" data-reactroot="">
                                    <div class="pb pb-free-item" data-reactid="2"><a data-reactid="3" data-track="included-free" href="#bonus-content" role="button"><span class="pb-free-item-label" data-reactid="4">Included Free:</span><!-- react-text: 5 --> <!-- /react-text --><span class="pb-free-item-link" data-reactid="6">1 item</span></a>
                                    </div>
                                </div>
                            </div>
                        </div>
                        <div class="item-button-wrapper">
                            <div class="cart-button" data-button-state-id="ADD_TO_CART" data-purchasable="true" data-service-down="false"><a class="btn btn-secondary btn-block btn-sm " href="#" role="button"><span class="ficon-cart"></span> <span class="label-add-to-cart">Add to Cart</span></a>
                            </div>
                            <div class="cart-service-down-modal-container"></div>
                        </div>
                        <div class="root-sc-notify-in-stock"></div>
                    </div>
                    <div class="eco-rebates-wrapper"></div>
                </div>
                <div class="list-item-open-box-hero-price-content"></div>
            </div>
        </div>
    </div>
</div>

In [15]:
#ahora, con la idea de preparar un bucle, meto el resultado del primer laptop en una variable llamada container.
container = containers[0]

In [16]:
#dandole distintos valores de etiqueta a container me dara aceso a distintas partes del codigo, por ejemplo h4 y label, voy 
#recorriendo el bloque del ordenador y observando donde esta lo que quiero descargar
container.h4

<h4><a data-rank="pdp" href="/site/dell-inspiron-15-6-touch-screen-laptop-intel-core-i5-8gb-memory-2tb-hard-drive-black/6097909.p?skuId=6097909"><span class="new-product">New!</span> Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black</a></h4>

In [17]:
container.label

<label for="add-to-compare-6097909"><input autocomplete="off" class="list-item-compare-checkbox sr-only" id="add-to-compare-6097909" type="checkbox"><div aria-hidden="true" class="checkbox-brand"><i class="custom-input"></i></div>Add to Compare</input></label>

In [18]:
#estoy interesado en obtener la valoracion de cada ordenador, su marca, una imagen, sus especificaciones y el precio, 
#veamoslo mas abajo

In [19]:
rating = container["data-average-rating"]
rating

u'3.4'

In [20]:
brand = container["data-brand"]
brand

u'{"brand":"Dell"}'

In [21]:
image = container["data-img-path"]
image

u'BestBuy_US/images/products/6097/6097909_sa.jpg'

In [22]:
specs = container["data-name"]
specs 

u'Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black'

In [23]:
price = container["data-price"]
price

u'529.99'

In [30]:
filename ="laptops.csv"  #defino un archivo donde guardare el resultado del web scraping
f = open(filename, "w")
#los headers de mi archivo csv
headers = "brand", "rating", "price", "specs", "image\n"
#f.write("headers")


#recordemos el valor de containers: containers =  page_soup.findAll("div" ,{"class":"list-item"})
#es decir nos va a dar todos los bloques en la web que tengan las etiquetas div y class=list-item
#es decir, nos va  adar todos los laptop de la pagina

for container in containers:
    brand = container["data-brand"].encode('ascii', 'ignore')
    rating = container["data-average-rating"].encode('ascii', 'ignore')
    price = container["data-price"].encode('ascii', 'ignore')
    specs = container["data-name"].encode('ascii', 'ignore')
    image = container["data-img-path"].encode('ascii', 'ignore')
    print "la marca es %s: " %  brand
    print "la valoracion es %s: " % rating
    print "las especificaciones son %s: " % specs
    print "la imagen es %s: " % image
    #print ("brand: " + brand ).encode('ascii', 'ignore')
    #print ("rating: " + rating ).encode('ascii', 'ignore')
    #print ("price: " + price ).encode('ascii', 'ignore')
    #print ("specs: " + specs ).encode('ascii', 'ignore')
    #print ("image: " + image ).encode('ascii', 'ignore')
    
    f.write( brand + "," + rating + "," + price + "," + specs + "," + image + "\n" )

f.close()

la marca es {"brand":"Dell"}: 
la valoracion es 3.4: 
las especificaciones son Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 2TB Hard Drive - Black: 
la imagen es BestBuy_US/images/products/6097/6097909_sa.jpg: 
la marca es {"brand":"HP"}: 
la valoracion es 4.3: 
las especificaciones son HP - 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 1TB Hard Drive - HP finish in jet black: 
la imagen es BestBuy_US/images/products/5868/5868705_sa.jpg: 
la marca es {"brand":"Dell"}: 
la valoracion es 4.3: 
las especificaciones son Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i3 - 8GB Memory - 1TB Hard Drive - Black: 
la imagen es BestBuy_US/images/products/5835/5835852_sa.jpg: 
la marca es {"brand":"Microsoft"}: 
la valoracion es 4.8: 
las especificaciones son Microsoft - Surface Pro Core M with Black Type Cover - Platinum: 
la imagen es BestBuy_US/images/products/6111/6111206_ra.jpg: 
la marca es {"brand":"Apple"}: 
la valoracion es 4.8: 
las especifi

In [ ]:
#esto me genera un archivo csv donde quedan almacenadas las caracteristicas que queria obtener de cada laptop

### MEJORAS AL EJERCICIO

* no me saca los headers del archivo csv, a pesar de haberlos definido
* No soy capaz de hacer un crawl horizontal y saltar al resto de paginas. Mi archivo csv solo contiene los datos de la primera pagina. Se que tendria que hacer un for y modificar la url para anyadir lo que diferencia a cada pagina pero no me sale todo lo que he probado. Seria algo asi:

#url_salto = ['/pagina2', '/pagina3', '/pagina4', ........'/pagina10'] 

#for url in url_salto:
  #my_url = "https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001" + url_salto
  #.....................................................................
  #.....................................................................
  #.....................................................................


